In [1]:
#imports

import pandas as pd
import gensim
import pyLDAvis.gensim
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora, models
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint

pd.set_option('display.max_colwidth', -1)

## Classifying Tweets to Categories
<a id="intertopic_map"></a>

## LDA - Intertopic Distance Map

In [2]:
airlines = pd.read_csv('./data/with_sentiment.csv')

In [3]:
# Create Dictionary
id2word = corpora.Dictionary(airlines.lemmatized.str.split())

# Create Corpus
texts = airlines.lemmatized.str.split()

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [4]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=42,
                                           update_every=1,
                                           chunksize=500,
                                           passes=90,
                                           alpha='auto',
                                           per_word_topics=True)

In [5]:
# Print the keywords in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.012*"\'new\'," + 0.011*"\'seat\'," + 0.008*"\'first\'," + '
  '0.007*"\'ticket\'," + 0.007*"\'travel\'," + 0.005*"\'class\'," + '
  '0.005*"\'trip\'," + 0.005*"\'chicago\'," + 0.005*"\'free\'," + '
  '0.004*"\'via\']"'),
 (1,
  '0.011*"\'max\'," + 0.009*"\'american\'," + 0.009*"\'boeing\'," + '
  '0.007*"\'flight\'," + 0.006*"\'ever\'," + 0.006*"\'worst\'," + '
  '0.005*"\'delta\'," + 0.005*"\'far\'," + 0.005*"\'san\'," + '
  '0.005*"\'cancellation\',"'),
 (2,
  '0.009*"\'plane\'," + 0.008*"\'service\'," + 0.008*"\'get\'," + '
  '0.008*"\'customer\'," + 0.007*"\'time\'," + 0.007*"\'hour\'," + '
  '0.007*"\'u\'," + 0.007*"\'bag\'," + 0.006*"\'one\'," + 0.005*"\'day\',"')]


In [6]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.246682  0.009738  1       1        64.266563
0      0.112825 -0.176197  2       1        18.432348
1      0.133857  0.166459  3       1        17.301085, topic_info=     Category        Freq             Term       Total  loglift  logprob
126   Default  110.000000  'new',           110.000000  30.0000  30.0000
113   Default  104.000000  'seat',          104.000000  29.0000  29.0000
2156  Default  98.000000   'max',           98.000000   28.0000  28.0000
556   Default  81.000000   'american',      81.000000   27.0000  27.0000
7608  Default  78.000000   'boeing',        78.000000   26.0000  26.0000
89    Default  72.000000   'first',         72.000000   25.0000  25.0000
277   Default  284.000000  'plane',         284.000000  24.0000  24.0000
448   Default  276.000000  'service',       276.000000  23.0000  23.0000
96    Default  116.000000  'ticket',        116.000000  22.0000  22.0000
59    Default  273.000000  'get',           273.000000  21.0000  21.0000
100   Default  259.000000  'customer',      259.000000  20.0000  20.0000
382   Default  61.000000   'travel',        61.000000   19.0000  19.0000
363   Default  242.000000  'time',          242.000000  18.0000  18.0000
225   Default  236.000000  'hour',          236.000000  17.0000  17.0000
713   Default  53.000000   'ever',          53.000000   16.0000  16.0000
221   Default  222.000000  'bag',           222.000000  15.0000  15.0000
50    Default  228.000000  'u',             228.000000  14.0000  14.0000
1266  Default  49.000000   'worst',         49.000000   13.0000  13.0000
579   Default  48.000000   'delta',         48.000000   12.0000  12.0000
87    Default  49.000000   'class',         49.000000   11.0000  11.0000
979   Default  46.000000   'far',           46.000000   10.0000  10.0000
704   Default  46.000000   'trip',          46.000000   9.0000   9.0000 
690   Default  44.000000   'san',           44.000000   8.0000   8.0000 
1057  Default  46.000000   'chicago',       46.000000   7.0000   7.0000 
1572  Default  45.000000   'free',          45.000000   6.0000   6.0000 
1708  Default  62.000000   'wrong',         62.000000   5.0000   5.0000 
127   Default  180.000000  'one',           180.000000  4.0000   4.0000 
7632  Default  41.000000   'cancellation',  41.000000   3.0000   3.0000 
3194  Default  41.000000   'landing',       41.000000   2.0000   2.0000 
71    Default  243.000000  'flight',        243.000000  1.0000   1.0000 
...       ...         ...        ...               ...     ...      ... 
3194  Topic3   40.670017   'landing',       41.261452   1.7400  -5.3705 
9397  Topic3   35.609894   'seaworld',      36.221542   1.7374  -5.5034 
7634  Topic3   29.514769   ['extends',      30.073284   1.7357  -5.6912 
8175  Topic3   30.869627   'spiderman',     31.543457   1.7328  -5.6463 
5600  Topic3   26.835060   'southwest',     27.453110   1.7316  -5.7863 
3067  Topic3   23.708296   'worker',        24.286083   1.7303  -5.9102 
1386  Topic3   22.742254   'diego',         23.309242   1.7298  -5.9518 
752   Topic3   35.317360   'much',          36.202782   1.7296  -5.5117 
1543  Topic3   25.470554   'major',         26.115099   1.7294  -5.8385 
1276  Topic3   24.456041   'big',           25.099358   1.7284  -5.8792 
994   Topic3   26.848242   'wifi',          27.582325   1.7274  -5.7858 
7018  Topic3   21.803513   'minor',         22.414921   1.7267  -5.9940 
2460  Topic3   19.760931   ['b',            20.323912   1.7263  -6.0923 
7653  Topic3   19.973413   'billion',       20.551178   1.7259  -6.0816 
1098  Topic3   22.666229   'food',          23.322256   1.7259  -5.9552 
2515  Topic3   24.639565   'coming',        25.358669   1.7256  -5.8717 
7725  Topic3   19.092781   'college',       19.652872   1.7255  -6.1267 
577   Topic3   23.380638   'attendant',     24.066923   1.7255  -5.9241 
3719  Topic3   23.076048   'country',

## Doc2Vec

In [7]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(airlines['text'])]

#  building a model
model = Doc2Vec(documents, vector_size=300, window=5, min_count=1, dm =1)
#Infer vector for a new document:
vector = model.infer_vector(["system", "response"])
#vector

In [8]:
similar_doc = model.docvecs.most_similar(3,topn = 50)
similar_doc.sort(key= lambda x: x[1])
similar_doc

[(5704, 0.8619507551193237),
 (1603, 0.8625760078430176),
 (5347, 0.8648976683616638),
 (3007, 0.8652684092521667),
 (913, 0.865969181060791),
 (1885, 0.8667536973953247),
 (1429, 0.8668698072433472),
 (3896, 0.8671457767486572),
 (2494, 0.867577314376831),
 (5398, 0.8688647747039795),
 (3182, 0.8700319528579712),
 (655, 0.8701715469360352),
 (4844, 0.870286762714386),
 (1975, 0.8714767694473267),
 (921, 0.8716994524002075),
 (3727, 0.872498631477356),
 (304, 0.8727860450744629),
 (490, 0.8728874325752258),
 (2002, 0.8739687204360962),
 (183, 0.8748645782470703),
 (5078, 0.8754234313964844),
 (2033, 0.8755291700363159),
 (2457, 0.8757647275924683),
 (5629, 0.8760643005371094),
 (2112, 0.8770704865455627),
 (1500, 0.8775063753128052),
 (629, 0.8776907324790955),
 (5309, 0.8779200315475464),
 (2999, 0.8786770105361938),
 (1240, 0.87958824634552),
 (5928, 0.8821991682052612),
 (422, 0.882508397102356),
 (5137, 0.882559061050415),
 (4823, 0.8825976848602295),
 (5627, 0.8835605978965759),
 

In [9]:
airlines.iloc[[i[0] for i in similar_doc],:]

,Unnamed: 0,text,tweet_coord,tweet_created,tweet_location,negative,positive,neutral,compound,lemmatized,clean,blob_polarity,blob_subjectivity
5704,5704,United Airlines put an underage passenger on a plane to the wrong country prompting an allnight ordeal between a,NaN,Mon Jul 01 21:32:31 +0000 2019,NaN,0.148,0.134,0.718,-0.0772,"['put', 'underage', 'passenger', 'plane', 'wrong', 'country', 'prompting', 'allnight', 'ordeal']",put underage passenger plane wrong country prompting allnight ordeal,-0.500000,0.900000
1603,1603,it would be great to get a callback from the agent that disconnected me trying to put me on hold after waiting to connectfor 60min,NaN,2015-02-21 21:08:56 -0800,"Chicago, IL",0.000,0.151,0.849,0.6249,"['would', 'great', 'get', 'callback', 'agent', 'disconnected', 'trying', 'put', 'hold', 'waiting', 'connectfor', 'min']",would great get callback agent disconnected trying put hold waiting connectfor min,0.800000,0.750000
5347,5347,United airlines looking for a database engineer and I dont know nobody I can recommend FML,NaN,Thu Jul 11 22:13:22 +0000 2019,Ikeja,0.000,0.325,0.675,0.6486,"['looking', 'database', 'engineer', 'dont', 'know', 'nobody', 'recommend', 'fml']",looking database engineer dont know nobody recommend fml,0.000000,0.000000
3007,3007,yes when I got to the gate I specifically asked if there where any other seats Very discouraging to walk past the crew 37d,NaN,2015-02-18 22:27:14 -0800,NaN,0.123,0.104,0.772,-0.1263,"['yes', 'got', 'gate', 'specifically', 'asked', 'seat', 'discouraging', 'walk', 'past', 'crew']",yes got gate specifically asked seat discouraging walk past crew,-0.250000,0.250000
913,913,is it even legal for you guys to advertise flights that you cant honor,NaN,2015-02-22 21:03:42 -0800,H-town,0.163,0.093,0.744,-0.2796,"['even', 'legal', 'guy', 'advertise', 'flight', 'cant', 'honor']",even legal guy advertise flight cant honor,0.200000,0.200000
1885,1885,troubling news time to raise the bar on legroom,NaN,2015-02-21 12:14:34 -0800,"ÜT: 39.768182,-86.167261",0.304,0.000,0.696,-0.5423,"['troubling', 'news', 'time', 'raise', 'bar', 'legroom']",troubling news time raise bar legroom,0.000000,0.000000
1429,1429,flight delayed by 24 hours lost my wallet have no money to eat or sleep yet your representatives will not helpunitedsucks,NaN,2015-02-22 07:03:06 -0800,NaN,0.262,0.000,0.738,-0.6597,"['delayed', 'hour', 'lost', 'wallet', 'money', 'eat', 'sleep', 'yet', 'representative', 'helpunitedsucks']",delayed hour lost wallet money eat sleep yet representative helpunitedsucks,0.000000,0.000000
3896,3896,When Spirit Airlines stock is 16 higher than your own you should really reconsider your continued p,NaN,Wed Jun 19 23:40:26 +0000 2019,NaN,0.000,0.102,0.898,0.1779,"['spirit', 'stock', 'higher', 'really', 'reconsider', 'continued', 'p']",spirit stock higher really reconsider continued p,0.225000,0.350000
2494,2494,no because you will charge me or delay me further United 1612 still waiting,NaN,2015-02-20 07:42:57 -0800,"Orlando, FL",0.246,0.153,0.601,-0.1779,"['charge', 'delay', 'still', 'waiting']",charge delay still waiting,0.000000,0.000000
5398,5398,Southwest and United Airlines extend 737 Max cancellations after new flaw found via,NaN,Sun Jul 14 21:03:19 +0000 2019,NaN,0.000,0.290,0.710,0.5423,"['southwest', 'extend', 'max', 'cancellation', 'new', 'flaw', 'found', 'via']",southwest extend max cancellation new flaw found via,0.136364,0.454545
